This file is to compile all (at least some) excels of 共同利用研究者連絡先（公募利用）and merge all contacts into one list.
Arthur Li, 2023
# TODO list
- read H25,H26 and merge to list

In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
# configurations
path = "../data/共同利用研究者連絡先（公募利用）"
# skipping 3 rows for H25,26 (old format)
skip_H26 = 3
# skipping 2 rows for new format
skip = 2
# config columns to use
cols = ['氏名','所属','職名', 'メールアドレス']

In [3]:
class fileClass:
    def __init__(self,path,fn):
        self.path = path
        self.fn = fn
        return

    def readFile(self, skiprows):
        self.dt = pd.read_excel(f"{self.path}/{self.fn}", skiprows=skiprows)
        # remove FULL CHARACTER space in the column names
        self.dt.columns = self.dt.columns.str.replace("　", "")
        return self.dt

    def selectData(self, cols):
        self.dt_select = self.dt[cols]
        for i,col in enumerate(cols):
            ## remove \n and space in affliation
            self.dt_select.iloc[:,i] = self.dt_select[col].str.replace("　", "")
            self.dt_select.iloc[:,i] = self.dt_select[col].str.replace(" ", "")
            self.dt_select.iloc[:,i] = self.dt_select[col].str.replace("\n", "・")
        return self.dt_select
    
    def printAll(self):
        print(self.dt)
        return
    
    def printCols(self):
        print(self.dt.columns)
        return


def loopAndStack(path, fn_list_new, fn_list_old, skiprows, cols):
    def LASOldNew(path, fn_list, skiprows, cols, status):
        # config old and new skips
        skip_old, skip_new = skiprows
        # initialize first file and list_all
        fn0 = fn_list[0]
        file = fileClass(path=path,fn=fn0)   
        file.readFile(skiprows=skip_new if status=='new' else skip_old)
        list_all = file.selectData(cols=cols)
        for fn in fn_list[1:]:
            print(fn)
            # loop over all files and select data
            file = fileClass(path=path,fn=fn)
            file.readFile(skiprows=skip_new if status=='new' else skip_old)
            # if H25: not email address but Unnamed: 10
            if fn[1:3]=='25':
                list_toup = file.selectData(cols=np.append(cols[:-1],'Unnamed: 10'))
            else:
                list_toup = file.selectData(cols=cols)
            # stack all: note that they are redundant now
            list_all = np.vstack((list_all,list_toup))
        return list_all
    # a function to stack all infomation (with redundence)
    old = LASOldNew(path=path, fn_list=fn_list_old, skiprows=skiprows, cols=cols, status='old')
    new = LASOldNew(path=path, fn_list=fn_list_new, skiprows=skiprows, cols=cols, status='new')
    # stack both
    list_all = np.vstack((old,new))
    return list_all

def removeRedundent(list_all):
    # a function to remove redundent names
    new_list = list_all[0:2,:]
    na = 0
    redun = 0
    # then loop every row to check if name exist
    for i,row in enumerate(list_all[1:,:]):
        if row[0] in new_list[:,0]:
            # if name already exist, pass
            redun += 1
            pass
        else:
            new_list = np.vstack((new_list,row))
    # show info of removed redundency
    print(f"Org: {len(list_all)}; After: {len(new_list)}")
    print(f"Redundency: {redun}")
    return new_list

# a function return old and new list
def selectOldNew(file_list):
    old = []
    new = []
    for fn in file_list:
        # in all files, if it is < H26
        if fn[0]=='H' and int(fn[1:3]) <= 26:
            old = np.append(old,fn)
        elif  (fn[0]=='Ｈ' or fn[0]=='R') or int(fn[1:3]) > 26:
            new = np.append(new,fn)
    return old,new

In [4]:
# reading all filenames
file_list = os.listdir(path=path)
print(file_list)
old_file_list,new_file_list = selectOldNew(file_list)
print(f"Old: {old_file_list}")
print(f"New: {new_file_list}")

['H26年度（大槌地区）.xlsx', 'H27 大槌・外来.xlsx', 'H28 大槌・外来.xlsx', 'H29 大槌・外来.xlsx', 'H30 大槌・外来.xlsx', 'R1大槌・外来.xlsx', 'R2 大槌・外来.xlsx', 'R3 大槌・外来.xlsx', 'R4 大槌・外来.xlsx', 'R5 大槌・外来.xlsx', 'Ｈ25年度（大槌地区）.xlsx']
Old: ['H26年度（大槌地区）.xlsx']
New: ['H27 大槌・外来.xlsx' 'H28 大槌・外来.xlsx' 'H29 大槌・外来.xlsx' 'H30 大槌・外来.xlsx'
 'R1大槌・外来.xlsx' 'R2 大槌・外来.xlsx' 'R3 大槌・外来.xlsx' 'R4 大槌・外来.xlsx'
 'R5 大槌・外来.xlsx' 'Ｈ25年度（大槌地区）.xlsx']


In [5]:
# Testing with one file: new format
print("testing one case")
fn0 = new_file_list[0]
print(new_file_list)
print(fn0)
file = fileClass(path=path,fn=fn0)
file.readFile(skiprows=skip)
results = file.selectData(cols=cols)
############################################
#print(results)

testing one case
['H27 大槌・外来.xlsx' 'H28 大槌・外来.xlsx' 'H29 大槌・外来.xlsx' 'H30 大槌・外来.xlsx'
 'R1大槌・外来.xlsx' 'R2 大槌・外来.xlsx' 'R3 大槌・外来.xlsx' 'R4 大槌・外来.xlsx'
 'R5 大槌・外来.xlsx' 'Ｈ25年度（大槌地区）.xlsx']
H27 大槌・外来.xlsx


C:\Users\Arthurli\anaconda3\lib\site-packages\pandas\core\indexing.py:1783: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [6]:
# Testing with one file: check H25
print("testing one case")
fn0 = new_file_list[-1]
print(new_file_list)
print(fn0)
file = fileClass(path=path,fn=fn0)
file.readFile(skiprows=skip)
#file.printAll()
file.printCols()
results = file.selectData(cols=np.append(cols[:-1],'Unnamed: 10'))
############################################
#print(results)

testing one case
['H27 大槌・外来.xlsx' 'H28 大槌・外来.xlsx' 'H29 大槌・外来.xlsx' 'H30 大槌・外来.xlsx'
 'R1大槌・外来.xlsx' 'R2 大槌・外来.xlsx' 'R3 大槌・外来.xlsx' 'R4 大槌・外来.xlsx'
 'R5 大槌・外来.xlsx' 'Ｈ25年度（大槌地区）.xlsx']
Ｈ25年度（大槌地区）.xlsx
Index(['No.', '受付番号', '所属', '職名', '氏名', '研究題目', '利用日程', '利用予定者数', 'センター担当者',
       'Unnamed: 9', 'Unnamed: 10'],
      dtype='object')


In [7]:
# read and save all: some names are repeated
results = loopAndStack(path=path, fn_list_new=new_file_list, fn_list_old=old_file_list,skiprows=(skip_H26,skip), cols=cols)
print("Show full list (redundant): ")
contact_list_redun = pd.DataFrame(results, columns=cols)
contact_list_redun = contact_list_redun.dropna()
fn = 'contact_list_redun.xlsx'
contact_list_redun.to_excel(fn)
#########################################
#print(contact_list_redun)

H28 大槌・外来.xlsx
H29 大槌・外来.xlsx
H30 大槌・外来.xlsx
R1大槌・外来.xlsx
R2 大槌・外来.xlsx
R3 大槌・外来.xlsx
R4 大槌・外来.xlsx
R5 大槌・外来.xlsx
Ｈ25年度（大槌地区）.xlsx
Show full list (redundant): 


In [8]:
# to remove redundency
contact_list = removeRedundent(results)
print("Show new list (clean): ")
contact_list = pd.DataFrame(contact_list,columns=cols)
contact_list = contact_list.dropna()
######################################
#print(contact_list)

Org: 390; After: 115
Redundency: 276
Show new list (clean): 


In [9]:
# save the contact list
contact_list.to_excel('contact_list.xlsx')

# Then, assign email address 

In [10]:
def checkAndAssign(df, contact_list,checking_by='氏名'):
    # a loop function that checks name and assign email
    no_name = []
    for i,name in enumerate(df['PI-name']):
        if name in np.array(contact_list.iloc[:,0]):
            # assign email address: at col 7
            df.iloc[i,7] = str(np.array(contact_list[np.array(contact_list[checking_by])==name])[0][-1])
        elif name not in no_name:
            # if name not recorded, put into no_name list
            no_name = np.append(no_name,name)
        else:
            continue
    return df, no_name

In [11]:
# specify file to read, and check its column names
obs_fn = "ObservationLog_daily.xlsx"
df = pd.read_excel(obs_fn)
print(df.columns)
contact_fn = "contact_list.xlsx"
contact_list = pd.read_excel(contact_fn)
print(contact_list.columns)

Index(['yyyy/mm/dd', 'Ship-name', 'PI-name', 'Pl-affiliation', 'PI-position',
       'Instruments', 'CTD (0/1)', 'PI-email', 'Unnamed: 8', 'Ship-name list'],
      dtype='object')
Index(['Unnamed: 0', '氏名', '所属', '職名', 'メールアドレス'], dtype='object')


In [12]:
# assign email address to new df
new_df,no_exist_name = checkAndAssign(df,contact_list)
####################################################
#print(new_df)
#print(no_exist_name)

C:\Users\Arthurli\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """


In [13]:
# save as temporary excel file -> copy and update on drive
new_df.to_excel('tmp.xlsx')

End of file